In [4]:
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime
from zoneinfo import ZoneInfo

# -------------------------
# LOAD DATA
# -------------------------
df = pd.read_csv("googleplaystore.csv")

# -------------------------
# DATA CLEANING
# -------------------------

# Clean Installs column
df["Installs"] = (
    df["Installs"]
    .astype(str)
    .str.replace("+", "", regex=False)
    .str.replace(",", "", regex=False)
)
df["Installs"] = pd.to_numeric(df["Installs"], errors="coerce")

# Convert Reviews and Rating
df["Reviews"] = pd.to_numeric(df["Reviews"], errors="coerce")
df["Rating"] = pd.to_numeric(df["Rating"], errors="coerce")

# Convert Size to MB
def size_to_mb(x):
    try:
        if isinstance(x, str):
            x = x.strip()
            if "M" in x:
                return float(x.replace("M", ""))
            elif "k" in x:
                return float(x.replace("k", "")) / 1024
        return None
    except:
        return None

df["Size_MB"] = df["Size"].apply(size_to_mb)

# Convert Last Updated to datetime
df["Last Updated"] = pd.to_datetime(df["Last Updated"], errors="coerce")

# -------------------------
# FILTER CONDITIONS
# -------------------------

filtered = df[
    (df["Rating"] >= 4.0) &
    (df["Size_MB"] >= 10) &
    (df["Last Updated"].dt.month == 1)
]

# -------------------------
# TOP 10 CATEGORIES BY INSTALLS
# -------------------------

top_categories = (
    filtered.groupby("Category")["Installs"]
    .sum()
    .sort_values(ascending=False)
    .head(10)
    .index
)

top_df = filtered[filtered["Category"].isin(top_categories)]

# -------------------------
# AGGREGATED SUMMARY
# -------------------------

summary = (
    top_df.groupby("Category")
    .agg(
        Average_Rating=("Rating", "mean"),
        Total_Reviews=("Reviews", "sum")
    )
    .reset_index()
)

# -------------------------
# TIME CONDITION (3PM–5PM IST)
# -------------------------

current_time = datetime.now(ZoneInfo("Asia/Kolkata"))
current_hour = current_time.hour

print("Current IST Time:", current_time.strftime("%Y-%m-%d %H:%M:%S"))

if 15 <= current_hour < 17 and not summary.empty:

    fig = go.Figure()

    fig.add_trace(go.Bar(
        x=summary["Category"],
        y=summary["Average_Rating"],
        name="Average Rating"
    ))

    fig.add_trace(go.Bar(
        x=summary["Category"],
        y=summary["Total_Reviews"],
        name="Total Reviews"
    ))

    fig.update_layout(
        title="Top 10 Categories (Jan Updated, Rating ≥4, Size ≥10MB)\nVisible Only Between 3 PM IST and 5 PM IST",
        barmode="group",
        xaxis_title="App Category",
        yaxis_title="Value",
        template="plotly_white"
    )

    fig.show()

else:
    print("⚠ Dashboard Locked")
    print("Graph is visible ONLY between 3 PM IST and 5 PM IST.")


Current IST Time: 2026-02-16 09:01:33
⚠ Dashboard Locked
Graph is visible ONLY between 3 PM IST and 5 PM IST.
